# QR decomposition

In [16]:
import numpy as np

In [17]:
import numpy.linalg as nla

In [18]:
A = np.array([[0.34,0.4,0.3,0.9],[0.45,0.25,0.98,0.46],[0.923,0.34,3,-2],[0.34,-0.45,0.1,0.55]])

In [19]:
Q = nla.qr(A)[0]
R = nla.qr(A)[1]

In [20]:
print(Q)
print(R)

[[-0.29986176 -0.45548197  0.72189882 -0.4260061 ]
 [-0.39687586 -0.16662497  0.24300203  0.86929609]
 [-0.81403649 -0.07202495 -0.52451613 -0.23883014]
 [-0.29986176  0.87154157  0.38038788 -0.07617926]]
[[-1.13385581 -0.36099828 -2.95099251  1.01071052]
 [ 0.         -0.64053122 -0.42885775  0.1368165 ]
 [ 0.          0.         -1.08079798  2.01973547]
 [ 0.          0.          0.          0.45223238]]


In [56]:
np.matmul(Q,R)

array([[ 0.34 ,  0.4  ,  0.3  ,  0.9  ],
       [ 0.45 ,  0.25 ,  0.98 ,  0.46 ],
       [ 0.923,  0.34 ,  3.   , -2.   ],
       [ 0.34 , -0.45 ,  0.1  ,  0.55 ]])

In [57]:
A

array([[ 0.34 ,  0.4  ,  0.3  ,  0.9  ],
       [ 0.45 ,  0.25 ,  0.98 ,  0.46 ],
       [ 0.923,  0.34 ,  3.   , -2.   ],
       [ 0.34 , -0.45 ,  0.1  ,  0.55 ]])

## Givens Rotation

In [23]:
def rotate_vector(x,y):
    ''' 
    Given a vector with two element x and y 
    rotate and normalize the vector
    '''
    if y == 0.0: 
        return 1.0,0.0
    r = np.hypot(x,y) 
    return x/r, -y/r

In [24]:
def givens_rotation(A):
    """Given a matrix A output Q and R matrix via
       Givens rotation """
    m, n = np.shape(A)

    # Initialize orthogonal matrix Q and upper triangular matrix R
    Q = np.identity(m)
    R = A

    # Iterate over lower triangular matrix.
    (rows, cols) = np.tril_indices(m, -1, n) # Return the indices for the lower-triangle of an array
    for (row, col) in zip(rows, cols):

        if R[row, col] != 0:
            (c, s) = rotate_vector(R[col, col], R[row, col])

            G = np.identity(m)
            G[[col, row], [col, row]] = c
            G[row, col] = s
            G[col, row] = -s

            R = np.dot(G, R)
            Q = np.dot(Q, np.transpose(G))

    return (Q, R)

In [25]:
A_mat = np.array([[0.34,0.4,0.3,0.9],[0.45,0.25,0.98,0.46],[0.923,0.34,3,-2],[0.34,-0.45,0.1,0.55]])
Q, R = givens_rotation(A_mat)
# A_mat = np.array([[1,2,3],[4,5,6],[7,8,9]])
# Q, R = givens_QR(A_mat)
print(Q)
print(R)

[[ 0.29986176  0.45548197 -0.72189882 -0.4260061 ]
 [ 0.39687586  0.16662497 -0.24300203  0.86929609]
 [ 0.81403649  0.07202495  0.52451613 -0.23883014]
 [ 0.29986176 -0.87154157 -0.38038788 -0.07617926]]
[[ 1.13385581e+00  3.60998282e-01  2.95099251e+00 -1.01071052e+00]
 [-2.18114435e-17  6.40531217e-01  4.28857747e-01 -1.36816497e-01]
 [ 4.50540381e-17 -1.75389979e-18  1.08079798e+00 -2.01973547e+00]
 [ 1.08369319e-17 -5.07920389e-18 -2.46348943e-18  4.52232385e-01]]


In [26]:
np.matmul(Q,R)

array([[ 0.34 ,  0.4  ,  0.3  ,  0.9  ],
       [ 0.45 ,  0.25 ,  0.98 ,  0.46 ],
       [ 0.923,  0.34 ,  3.   , -2.   ],
       [ 0.34 , -0.45 ,  0.1  ,  0.55 ]])

## Householder transformation

In [27]:
import numpy as np

In [28]:
def Householder(A):
    m,n = A.shape
    R = A
    Q = np.eye(m)

    for j in range(n):
        x = R[j:,j]
        e_1 = np.zeros(len(x))
        e_1[0] = 1
        v_k = np.sign(x[0])*nla.norm(x)*e_1+x
        v_k = v_k/nla.norm(v_k) # normalize the reflection vector

        # Form of Householder transformation is H = I - 2(vv^t)/(v^tv)
        # R[j:, :]H Will eliminate all subdiagonal elements
        # Hence R[j:, :]H = R[j:, :] - R[j:, :]*2(vv^t)/(v^tv)
        R[j:, :] = R[j:, :] - 2* np.outer(v_k, v_k).dot(R[j:, :])
        Q[:, j:] = Q[:, j:] - 2 * Q[:, j:].dot(np.outer(v_k, v_k))
    
    return(Q,R)

In [29]:
A = np.array([[0.34,0.4,0.3,0.9],[0.45,0.25,0.98,0.46],[0.923,0.34,3,-2],[0.34,-0.45,0.1,0.55]])
print(nla.cond(A))
Q,R=Householder(A)
print (Q)
print(R)

37.757292558597214
[[-0.29986176 -0.45548197  0.72189882  0.4260061 ]
 [-0.39687586 -0.16662497  0.24300203 -0.86929609]
 [-0.81403649 -0.07202495 -0.52451613  0.23883014]
 [-0.29986176  0.87154157  0.38038788  0.07617926]]
[[-1.13385581e+00 -3.60998282e-01 -2.95099251e+00  1.01071052e+00]
 [-5.42130523e-17 -6.40531217e-01 -4.28857747e-01  1.36816497e-01]
 [-1.18713320e-17  8.49702052e-19 -1.08079798e+00  2.01973547e+00]
 [ 1.22651845e-18  1.14436015e-16 -1.11022302e-16 -4.52232385e-01]]


In [30]:
np.transpose(Q)

array([[-0.29986176, -0.39687586, -0.81403649, -0.29986176],
       [-0.45548197, -0.16662497, -0.07202495,  0.87154157],
       [ 0.72189882,  0.24300203, -0.52451613,  0.38038788],
       [ 0.4260061 , -0.86929609,  0.23883014,  0.07617926]])

In [31]:
nla.inv(Q)

array([[-0.29986176, -0.39687586, -0.81403649, -0.29986176],
       [-0.45548197, -0.16662497, -0.07202495,  0.87154157],
       [ 0.72189882,  0.24300203, -0.52451613,  0.38038788],
       [ 0.4260061 , -0.86929609,  0.23883014,  0.07617926]])

In [22]:
R

array([[-1.13385581e+00, -3.60998282e-01, -2.95099251e+00,
         1.01071052e+00],
       [-5.42130523e-17, -6.40531217e-01, -4.28857747e-01,
         1.36816497e-01],
       [-1.18713320e-17,  8.49702052e-19, -1.08079798e+00,
         2.01973547e+00],
       [ 1.22651845e-18,  1.14436015e-16, -1.11022302e-16,
        -4.52232385e-01]])

In [32]:
np.matmul(Q,R)

array([[ 0.34 ,  0.4  ,  0.3  ,  0.9  ],
       [ 0.45 ,  0.25 ,  0.98 ,  0.46 ],
       [ 0.923,  0.34 ,  3.   , -2.   ],
       [ 0.34 , -0.45 ,  0.1  ,  0.55 ]])

## Eigenvalue Algorithms

for a traingular matrix

In [33]:
def eigValTriangularMatrix(A):
    """
    Given an upper or lower traingular matrix output eigenvalues of the matrix
    """
    
    return(np.diag(A))

In [34]:
A = np.array([[1,2,3],[0,3,4],[0,0,9]])

In [35]:
eigValTriangularMatrix(A)

array([1, 3, 9])

In [36]:
def eigValPowerIteration(A, x0, n_iter):
    x_1 = x0
    for i in range(n_iter):
        x_1 = np.dot(A,x_1)
        x_1 = x_1 / np.linalg.norm(x_1)
    
    eigVal = np.linalg.norm(np.dot(A, x_1))/np.linalg.norm(x_1)
    return x_1, eigVal
    

In [37]:
A = np.array([[1,2,3],[0,3,4],[0,0,9]])

In [38]:
eigValPowerIteration(A, [1,2,3], 100)

(array([0.4108907 , 0.50571163, 0.75856745]), 9.000000000000002)

In [ ]:
# QR Algorithm for eigen values 

In [39]:
def eigValQR_algorithm(A, n_iter):
    
    for i in range(n_iter):
        temp = nla.qr(A)
        Q = temp[0]
        R = temp[1]
        A = np.dot(R,Q)
    return(np.diag(A))


In [40]:
A = np.array([[5,2,0,0,0],[2,2,4,0,0],[0,4,4,2,0],[0,0,2,1,4],[0,0,0,4,5]])

In [41]:
eigValQR_algorithm(A, 100)

array([ 8.44883331,  7.03390237,  4.78830125, -2.60085857, -0.67017835])